# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [5]:
!pip install openai
!pip install python-dotenv

from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('KEY HERE')

In [85]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0.7,
        )

    return (response.choices[0].message.content)

In [86]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['Boxer','Wins','Losses','Salery'],# ENTER A TABLE COLUMNS HERE,
        'definition': ['name of the boxer','how often he won','how often he lost','his average salery']
        }# ENTER A TABLE DEFINITATIONS HERE
df = pd.DataFrame(data)
print(df)

    table          definition
0   Boxer   name of the boxer
1    Wins    how often he won
2  Losses   how often he lost
3  Salery  his average salery


In [87]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [88]:
print(text_tables)

Boxer: name of the boxer
Wins: how often he won
Losses: how often he lost
Salery: his average salery


In [89]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Read the question very carfully, word by word. Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [90]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Fetch the API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")
else: 
    print('all good')


all good


In [91]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="get me the top boxers")

In [92]:
print(return_OAI(pqt1))

```json
["Boxer"]
```


In [93]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="get me all the names of boxers in your table")

In [94]:
print(return_OAI(pqt3))

```json
["Boxer"]
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [95]:
pqt4 = prompt_question_tables.format(tables=text_tables,
                                     question="get me the average wins")
print(return_OAI(pqt4))

```json
{
    "tables": ["Wins"]
}
```


In [96]:
pqt5 = prompt_question_tables.format(tables=text_tables,
                                     question="get me the highest salery")
print(return_OAI(pqt5))

```json
["Boxer"]
```


In [97]:
pqt6 = prompt_question_tables.format(tables=text_tables,
                                     question="get me the average losses")
print(return_OAI(pqt6))

{
    "tables": ["Boxer"]
}


# Report:  

It did not work well. SOmehow the GPT model confused the tables. I reemplemented them, improved the names and descriptions. Yet, it only imporved slightly the performence